In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from IPython.display import display, Markdown
from dotenv import load_dotenv

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0,
    streaming=True,
    extra_body={"enable_thinking": False},
)

# 分解复杂任务

In [2]:
complex_task = """
分析一家公司的财务健康状况，基于以下数据：
- 收入：1000万美元
- 净利润：200万美元
- 总资产：1500万美元
- 总负债：700万美元
- 经营活动现金流：300万美元
"""

decomposition_prompt = """
将“分析公司财务健康状况”的任务拆解为3个子任务。对于每个子任务，简要描述其应完成的内容。

任务：{task}

子任务：
1.
"""

prompt_template = PromptTemplate(
    template=decomposition_prompt, input_variables=["task"]
)
chain = prompt_template | llm
subtasks = chain.invoke({"task": complex_task}).content
display(Markdown(subtasks))

1. **评估盈利能力**  
   分析公司的盈利水平，通过计算净利润率（净利润 / 收入）来判断公司赚取利润的能力。使用提供的收入和净利润数据，了解公司在每单位收入中创造的利润是否健康。

2. **分析财务结构与偿债能力**  
   评估公司的资本结构和偿债风险，通过计算资产负债率（总负债 / 总资产）等指标，判断公司依赖债务的程度及其偿还能力。

3. **考察现金流状况**  
   检查公司经营活动产生的现金流是否足以支持日常运营和债务偿还，结合净利润与现金流比较，判断盈利质量及资金流动性是否稳健。

# 在提示中链式组织子任务

In [5]:
def analyze_profitability(revenue, net_income):
    """分析公司的盈利能力。

    参数：
        revenue (float): 公司收入。
        net_income (float): 公司净利润。

    返回：
        str: 关于公司盈利能力的分析。
    """
    prompt = """
    根据以下数据分析公司的盈利能力：
    - 收入：${revenue} 百万美元
    - 净利润：${net_income} 百万美元

    请计算利润率，并简要分析公司的盈利情况。
    """

    prompt_template = PromptTemplate(
        template=prompt, input_variables=["revenue", "net_income"]
    )
    chain = prompt_template | llm
    return chain.invoke({"revenue": revenue, "net_income": net_income}).content


def analyze_liquidity(total_assets, total_liabilities):
    """分析公司的流动性。

    参数：
        total_assets (float): 公司总资产。
        total_liabilities (float): 公司总负债。

    返回：
        str: 关于公司流动性的分析。
    """
    prompt = """
    根据以下数据分析公司的流动性：
    - 总资产：${total_assets} 百万美元
    - 总负债：${total_liabilities} 百万美元

    请计算流动比率，并简要分析公司的流动性状况。
    """

    prompt_template = PromptTemplate(
        template=prompt, input_variables=["total_assets", "total_liabilities"]
    )
    chain = prompt_template | llm
    return chain.invoke(
        {"total_assets": total_assets, "total_liabilities": total_liabilities}
    ).content


def analyze_cash_flow(cash_flow):
    """分析公司的现金流。

    参数：
        cash_flow (float): 公司经营活动产生的现金流。

    返回：
        str: 关于公司现金流状况的分析。
    """
    prompt = """
    根据以下数据分析公司的现金流状况：
    - 经营活动现金流：${cash_flow} 百万美元

    请简要分析公司的现金流健康状况。
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=["cash_flow"])
    chain = prompt_template | llm
    return chain.invoke({"cash_flow": cash_flow}).content


# 执行子任务链
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

display(Markdown("# 盈利能力分析:\n" + profitability_analysis))
display(Markdown("\n # 流动性分析:\n" + liquidity_analysis))
display(Markdown("\n # 现金流分析:\n" + cash_flow_analysis))

# 盈利能力分析:
我们可以使用**利润率（Profit Margin）** 来分析公司的盈利能力。利润率的计算公式为：

$$
\text{利润率} = \frac{\text{净利润}}{\text{收入}} \times 100\%
$$

根据你提供的数据：

- 收入 = $10 百万美元  
- 净利润 = $2 百万美元  

代入公式：

$$
\text{利润率} = \frac{2}{10} \times 100\% = 20\%
$$

---

### **简要分析：**

该公司的**利润率是 20%**，说明每产生 1 美元的收入，公司可以赚取 0.2 美元的净利润。

这表明公司的盈利情况相对健康，具有一定的盈利能力。不过，是否优秀还需结合行业标准来判断：

- 如果是零售、制造业等行业，20% 的利润率通常属于较好的水平；
- 如果是高利润行业（如软件或金融服务），20% 可能略低；
- 如果是低利润行业（如餐饮、物流），20% 则可能表现优异。

因此，**20% 的利润率是一个中等偏上的盈利水平**，但具体评价需参考行业平均水平和公司历史数据。


 # 流动性分析:
要计算**流动比率（Current Ratio）**，我们需要知道公司的**流动资产**和**流动负债**。但目前你只提供了**总资产**和**总负债**，没有分别给出流动资产和流动负债的数据。

---

### 如果假设所有资产都是流动资产、所有负债都是流动负债：

我们可以做一个**简化假设**来进行估算：

- **流动资产 = 总资产 = $15 百万美元**
- **流动负债 = 总负债 = $7 百万美元**

那么：

$$
\text{流动比率} = \frac{\text{流动资产}}{\text{流动负债}} = \frac{15}{7} ≈ 2.14
$$

---

### 流动比率分析：

- **流动比率 = 2.14**  
  表示公司每有 $1 的流动负债，就有 $2.14 的流动资产用于偿还。这说明公司在短期内具有良好的偿债能力。
  
- **一般标准**：
  - 流动比率 **大于 1**：公司具备短期偿债能力。
  - 流动比率 **在 1.5 到 2.5 之间**：通常被认为是健康的范围。
  - 流动比率 **小于 1**：可能面临短期偿债压力。

---

### 结论：

如果假设所有资产和负债均为流动项，则该公司的流动比率为 **2.14**，属于健康水平，表明公司具有较强的短期偿付能力。然而，请注意这个结论基于一个简化的假设。实际分析中应使用详细的资产负债表数据（区分流动与非流动项目）以提高准确性。


 # 现金流分析:
根据提供的数据，公司的 **经营活动现金流为 $3 百万美元**。这是一个重要的指标，反映了公司通过主营业务活动产生的现金流入情况。

### 简要分析如下：

- **正向现金流**：$300 万美元的正经营现金流表明公司能够通过其核心业务产生足够的现金，这是健康的财务状况的表现。
- **运营能力良好**：说明公司具备良好的运营能力和盈利质量，能够在不依赖外部融资的情况下维持日常运营和投资需求。
- **可持续性信号**：如果这种正现金流是持续性的，通常意味着公司具有较强的盈利能力和市场竞争力。

### 注意事项：
虽然经营现金流为正值是一个积极信号，但还需结合其他信息（如投资活动现金流、筹资活动现金流、净利润、负债水平等）进行全面判断。例如：

- 如果公司正在大规模投资或扩张，可能需要额外的资金支持；
- 如果筹资现金流为负，说明公司在偿还债务或回购股份；
- 若净利润远高于经营现金流，可能存在大量非现金收入或应收账款问题。

### 总结：
基于现有信息，该公司的 **现金流健康状况良好**，尤其是经营活动能持续产生正现金流，是公司稳健运营的重要标志。

# 整合分析结果

In [6]:
def integrate_results(profitability, liquidity, cash_flow):
    """整合子任务的分析结果，提供整体评估。

    参数：
        profitability (str): 盈利能力分析。
        liquidity (str): 流动性分析。
        cash_flow (str): 现金流分析。

    返回：
        str: 公司整体财务健康状况的分析结果。
    """
    prompt = """
    根据以下分析内容，给出公司整体财务健康状况的评估：

    盈利能力分析：
    {profitability}

    流动性分析：
    {liquidity}

    现金流分析：
    {cash_flow}

    请总结关键要点，并整体评价公司的财务状况。
    """
    prompt_template = PromptTemplate(
        template=prompt, input_variables=["profitability", "liquidity", "cash_flow"]
    )
    chain = prompt_template | llm
    return chain.invoke(
        {"profitability": profitability, "liquidity": liquidity, "cash_flow": cash_flow}
    ).content


overall_analysis = integrate_results(
    profitability_analysis, liquidity_analysis, cash_flow_analysis
)
display(Markdown("# 公司整体财务健康状况分析:\n" + overall_analysis))

# 公司整体财务健康状况分析:
根据你提供的**盈利能力、流动性与现金流**三个方面的分析，以下是公司整体财务健康状况的总结和评估：

---

## ✅ **关键要点总结**

### 1. **盈利能力：**
- **利润率 = 20%**
- 表明每产生 $1 的收入，公司可以获得 $0.2 的净利润。
- 盈利能力良好，属于中等偏上水平。
- 是否优秀需结合行业标准判断，但总体来看具备较强的盈利质量。

### 2. **流动性：**
- **流动比率 ≈ 2.14（基于简化假设）**
- 每 $1 流动负债对应 $2.14 流动资产，短期偿债能力良好。
- 处于健康的流动性区间（1.5 - 2.5），表明公司具备足够的流动资产来覆盖短期债务。

### 3. **现金流：**
- **经营活动现金流 = $3 百万美元**
- 正向且稳定的经营现金流说明公司主营业务运行良好。
- 不依赖外部融资即可维持运营，具备良好的自我造血能力。

---

## 📊 **综合财务健康状况评价**

| 财务维度 | 状况 | 评价 |
|----------|------|------|
| 盈利能力 | 20% 利润率 | 良好，具备较强盈利能力 |
| 流动性   | 流动比率 ≈ 2.14 | 健康，短期偿债能力强 |
| 现金流   | 经营活动现金流为正 | 稳健，运营质量高 |

---

## 🟩 **总体评估结论：**

该公司的**财务状况整体健康、稳健**。  
- **盈利能力良好**，能够从其业务活动中获取较高利润；
- **流动性充足**，具备较强的短期偿债能力；
- **现金流稳定**，说明公司不仅有利润，还能将利润转化为实际现金，支持日常运营和未来发展。

---

## ⚠️ **注意事项与建议：**

虽然目前数据表现良好，但在进一步分析时仍需注意以下几点：

1. **行业对比**：利润率是否优于同行业平均水平？不同行业的盈利标准差异较大。
2. **非流动资产/负债的影响**：如果公司有大量固定资产或长期负债，应重新计算流动比率以更准确反映流动性。
3. **现金流结构**：观察投资活动和筹资活动现金流，了解公司是否在扩张、分红或偿还债务。
4. **持续性验证**：单一年度数据可能无法反映趋势，建议使用多年财务数据进行趋势分析。

---

## ✅ **最终结论：**

> **该公司目前的财务健康状况良好，具备较强的盈利能力、稳健的流动性和积极的现金流表现，是一个值得关注的稳健型企业。**

如能提供更多详细财务报表数据（如资产负债表、现金流量表全项），可进行更深入的财务分析。